<a href="https://colab.research.google.com/github/quicksilverTrx/mining_of_massive_datasets/blob/main/colab_notebooks/Collaborative_Filtering_Colab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS246 - Colab 4
## Collaborative Filtering

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 66kB/s 
     |████████████████████████████████| 204kB 43.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=6c8330340d8b1a5b52efca7deb1e6bb05870dc36a436cc01feb9ad1bdbd423a8
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 30 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

Now we authenticate a Google Drive client to download the filea we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1QtPy_HuIMSzhtYllT3-WeM3Sqg55wK_D'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.training')

id='1ePqnsQTJRRvQcBoF2EhoPU8CU1i5byHK'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.test')

id='1ncUBWdI5AIt3FDUJokbMqpHD2knd5ebp'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.item')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [ ]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [ ]:
'''
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
    '''

### Data Loading

In this Colab, we will be using the [MovieLens dataset](https://grouplens.org/datasets/movielens/), specifically the 100K dataset (which contains in total 100,000 ratings from 1000 users on ~1700 movies).

We load the ratings data in a 80%-20% ```training```/```test``` split, while the ```items``` dataframe contains the movie titles associated to the item identifiers.

In [ ]:
schema_ratings = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("item_id", IntegerType(), False),
    StructField("rating", IntegerType(), False),
    StructField("timestamp", IntegerType(), False)])

schema_items = StructType([
    StructField("item_id", IntegerType(), False),
    StructField("movie", StringType(), False)])

training = spark.read.option("sep", "\t").csv("MovieLens.training", header=False, schema=schema_ratings)
test = spark.read.option("sep", "\t").csv("MovieLens.test", header=False, schema=schema_ratings)
items = spark.read.option("sep", "|").csv("MovieLens.item", header=False, schema=schema_items)

In [ ]:
training.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [ ]:
training.show()

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|      1|      1|     5|874965758|
|      1|      2|     3|876893171|
|      1|      3|     4|878542960|
|      1|      4|     3|876893119|
|      1|      5|     3|889751712|
|      1|      7|     4|875071561|
|      1|      8|     1|875072484|
|      1|      9|     5|878543541|
|      1|     11|     2|875072262|
|      1|     13|     5|875071805|
|      1|     15|     5|875071608|
|      1|     16|     5|878543541|
|      1|     18|     4|887432020|
|      1|     19|     5|875071515|
|      1|     21|     1|878542772|
|      1|     22|     4|875072404|
|      1|     25|     4|875071805|
|      1|     26|     3|875072442|
|      1|     28|     4|875072173|
|      1|     29|     1|878542869|
+-------+-------+------+---------+
only showing top 20 rows



In [40]:
items.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- movie: string (nullable = true)



### Your task

Let's compute some stats!  What is the number of ratings in the training and test dataset? How many movies are in our dataset?

In [39]:
# YOUR CODE HERE
print(training.columns)
print(test.columns)
print(items.columns)
print(training.count())
print(test.count())
print(items.count())


['user_id', 'item_id', 'rating', 'timestamp']
['user_id', 'item_id', 'rating', 'timestamp']
['item_id', 'movie']
80000
20000
1682


In [38]:
ratings=training
# Count the total number of ratings in the dataset
numerator = ratings.select("rating").count()

# Count the number of distinct userIds and distinct movieIds
num_users = ratings.select("user_id").distinct().count()
num_movies = ratings.select("item_id").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  94.86% empty.


Using the training set, train a model with the Alternating Least Squares method available in the Spark MLlib: [https://spark.apache.org/docs/latest/ml-collaborative-filtering.html](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)

In [ ]:
# YOUR CODE HERE
type(training)
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql import Row
als = ALS( userCol="user_id", itemCol="item_id", ratingCol="rating",nonnegative = True, implicitPrefs = False, 
          coldStartStrategy="drop")
type(als)
#model = als.fit(training)



# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
             .build()
           # .addGrid(als.maxIter, [5, 50, 100, 200]) \
            

           
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  16


In [ ]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# Confirm cv was built
print(cv)


CrossValidator_cac64cf92ffa


In [17]:
model = cv.fit(training)

#Extract best model from the cv model above
best_model = model.bestModel

Now compute the RMSE on the test dataset.


In [20]:
# Print best_model
print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

# # Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())

# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

<class 'pyspark.ml.recommendation.ALSModel'>
**Best Model**
  Rank: 150
  MaxIter: 10
  RegParam: 0.1


In [21]:
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

0.9250428394704757


In [ ]:
# YOUR CODE HERE
'''
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))
'''

In [22]:
test_predictions.show()

+-------+-------+------+---------+----------+
|user_id|item_id|rating|timestamp|prediction|
+-------+-------+------+---------+----------+
|    251|    148|     2|886272547| 3.1448338|
|    332|    148|     5|887938486| 3.6800041|
|     44|    148|     4|878346946|  3.084648|
|    236|    148|     4|890117028| 2.6371062|
|    222|    148|     2|881061164| 2.8716118|
|    372|    148|     5|876869915| 3.7433248|
|    224|    148|     3|888104154| 3.3315296|
|      1|    148|     2|875240799| 2.6809747|
|    274|    148|     2|878946133| 3.4855423|
|    178|    148|     4|882824325| 3.5312593|
|    328|    148|     3|885048638| 3.2606215|
|    164|    148|     5|889402203|  4.124679|
|    120|    148|     3|889490499| 3.0956964|
|     92|    148|     2|877383934| 2.9589872|
|    447|    148|     4|878854729| 3.0888555|
|     15|    148|     3|879456049| 2.8378139|
|     59|    148|     3|888203175| 3.0451584|
|     49|    148|     1|888068195| 1.4459411|
|    130|    148|     4|876251127|

At this point, you can use the trained model to produce the top-K recommendations for each user.

In [24]:
# YOUR CODE HERE
# Generate top 5 movie recommendations for each user
userRecs = best_model.recommendForAllUsers(10)
# Generate top 5 user recommendations for each movie
movieRecs = best_model.recommendForAllItems(10)

In [26]:
test_predictions.columns

['user_id', 'item_id', 'rating', 'timestamp', 'prediction']

In [30]:
userRecs.limit(10).show()

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|    471|[{932, 4.5568876}...|
|    463|[{19, 4.3057923},...|
|    833|[{1597, 4.4862556...|
|    496|[{56, 4.3355064},...|
|    148|[{50, 4.67382}, {...|
|    540|[{1449, 4.8105183...|
|    392|[{483, 4.99175}, ...|
|    243|[{1449, 4.470333}...|
|    623|[{50, 4.4368787},...|
|    737|[{127, 4.6302605}...|
+-------+--------------------+



In [31]:

nrecommendations = userRecs\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('user_Id', col("rec_exp.item_id"), col("rec_exp.rating"))

In [32]:
nrecommendations.limit(10).show()


+-------+-------+---------+
|user_Id|item_id|   rating|
+-------+-------+---------+
|    471|    932|4.5568876|
|    471|    102|4.4632654|
|    471|      8| 4.360817|
|    471|   1167| 4.223188|
|    471|    477| 4.221383|
|    471|    140| 4.192864|
|    471|   1159|4.1861224|
|    471|    313|4.1542654|
|    471|    465|4.1212854|
|    471|    393| 4.121037|
+-------+-------+---------+



In [33]:
nrecommendations.join(items, on='item_id').filter('user_Id = 471').show()

+-------+-------+---------+--------------------+
|item_id|user_Id|   rating|               movie|
+-------+-------+---------+--------------------+
|    932|    471|4.5568876|    First Kid (1996)|
|    102|    471|4.4632654|Aristocats, The (...|
|      8|    471| 4.360817|         Babe (1995)|
|   1167|    471| 4.223188|Sum of Us, The (1...|
|    477|    471| 4.221383|      Matilda (1996)|
|    140|    471| 4.192864|Homeward Bound: T...|
|   1159|    471|4.1861224|      Stalker (1979)|
|    313|    471|4.1542654|      Titanic (1997)|
|    465|    471|4.1212854|Jungle Book, The ...|
|    393|    471| 4.121037|Mrs. Doubtfire (1...|
+-------+-------+---------+--------------------+



In [36]:
training.join(items, on='item_id').filter('user_Id = 471').sort('rating', ascending=False).limit(10).show()

+-------+-------+------+---------+--------------------+
|item_id|user_id|rating|timestamp|               movie|
+-------+-------+------+---------+--------------------+
|      8|    471|     5|889827881|         Babe (1995)|
|     82|    471|     5|889827822|Jurassic Park (1993)|
|     94|    471|     5|889828081|   Home Alone (1990)|
|    102|    471|     5|889828081|Aristocats, The (...|
|    140|    471|     5|889827918|Homeward Bound: T...|
|    225|    471|     5|889828026|101 Dalmatians (1...|
|    393|    471|     5|889827918|Mrs. Doubtfire (1...|
|    422|    471|     5|889827982|Aladdin and the K...|
|    465|    471|     5|889827822|Jungle Book, The ...|
|    477|    471|     5|889827918|      Matilda (1996)|
+-------+-------+------+---------+--------------------+



Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!